In [2]:
from Bio import SeqIO
from Bio.Seq import Seq

In [3]:
rs=Seq("GGCAACTTGGCCGAGTGGTTAAGGCGAAAGATTAGAAATCTTTTGGGCTTTGCCCGCGCAGGTTCGAGTCCTGCAGTTGTCG")
len(rs)

/home/f1lem0n/.miniconda3/envs/mgr/lib/python3.10/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('GNLAEWLRRKIRNLLGFARAGSSPAVV')

In [3]:
with open("../data/S288C_reference_genome_R62-1-1_20090218/rna_coding_R62-1-1_20090220.fasta") as f:
    nn_rec = list(SeqIO.parse(f, "fasta"))[5]
chr = "chr" + nn_rec.description.split("Chr")[1].split(" ")[1]
start, stop = nn_rec.description.split("from ")[1].split(",")[0].split("-")
start, stop = int(start), int(stop)
if start > stop:
    start, stop = stop - 1, start
chr, start, stop

('chrI', 182515, 182597)

In [5]:
with open("../data/S288C_reference_genome_R62-1-1_20090218/S288C_reference_sequence_R62-1-1_20090218.fsa") as f:
    ref = list(SeqIO.parse(f, "fasta"))[0]
print(ref.seq[start:stop].reverse_complement() == rs)

True


In [2]:
from Bio import Entrez, SeqIO
from io import StringIO

Entrez.email = "hajdylaf@gmail.com"

In [4]:
ids = ["GHM89747.1", "CAI4583015.1"]

aa_records = []
nn_records = []

for id in ids:
    with Entrez.efetch(db="protein", id=id, rettype="gb", retmode="text") as handle:
        gb = handle.read()
    aa_rec = SeqIO.read(StringIO(gb), "genbank").format("fasta")
    aa_rec = SeqIO.read(StringIO(aa_rec), "fasta")
    aa_records.append(aa_rec)
    cds_coords = gb.split("/coded_by=")[1].split("\n")[0]
    if "(" in cds_coords:
        cds_coords = cds_coords.split("(")[1].split(")")[0]
    seqid, coords = cds_coords.split(":")[0], cds_coords.split(":")[1]
    start, stop = coords.split("..")
    start, stop = int(start), int(stop)
    if start > stop:
        start, stop = stop, start
    start -= 1
    with Entrez.efetch(db="nucleotide", id=seqid, rettype="fasta", retmode="text") as handle:
        nn_rec = list(SeqIO.parse(handle, "fasta"))[0]
    nn_rec.seq = nn_rec.seq[start:stop]
    nn_records.append(nn_rec)

SeqIO.write(aa_records, "homologs_aa.fasta", "fasta")
SeqIO.write(nn_records, "pal2nal_guides.fasta", "fasta")


2

In [9]:
with open("../data/S288C_reference_genome_R62-1-1_20090218/S288C_reference_sequence_R62-1-1_20090218.fsa") as f:
    records = list(SeqIO.parse(f, "fasta"))
edited_records = []
for rec in records:
    try:
        new_id = "chr" + rec.description.split("chromosome=")[1].split("]")[0]
    except IndexError:
        new_id = "chrMito"
    rec.id = new_id
    rec.name = new_id
    rec.description = new_id + " " + rec.description
    edited_records.append(rec)
SeqIO.write(edited_records, "../data/S288C_reference_genome_R62-1-1_20090218/S288C_reference_sequence_R62-1-1_20090218_adj.fasta", "fasta")


17